In [2]:
import os
import json
from skimage import io
from skimage.transform import resize

image_folder = 'D:/Serba Serbi Kuliah/MATERI KULIAH SMT 5/AI/UAS_AI/Container-Detector-Use-KNN/data'
json_file = 'D:/Serba Serbi Kuliah/MATERI KULIAH SMT 5/AI/UAS_AI/Container-Detector-Use-KNN/data/annotations_color.json'


In [ ]:
def load_data_from_json_and_images(image_folder, json_file):
    images = []
    labels = []
    filenames = [] 
    
    with open(json_file) as f:
        data = json.load(f)

    # Membaca gambar dan label dari JSON
    for annotation in data['annotations']:
        image_filename = data['images'][annotation['image_id']]['file_name']  
        label = annotation['color_label']  
        
        # Memuat gambar dari folder
        image_path = os.path.join(image_folder, image_filename)
        image = io.imread(image_path)
        
        # Ubah ukuran gambar jika perlu (agar ukuran konsisten)
        image_resized = resize(image, (64, 64))  
        
        # Flatten gambar menjadi array 1D dengan list
        image_flattened = image_resized.flatten().tolist()  
        
        images.append(image_flattened)
        labels.append(label)
        filenames.append(image_filename) 
    
    return images, labels, filenames 

In [ ]:
def euclidean_distance(vec1, vec2):
    sum_squared = sum((a - b) ** 2 for a, b in zip(vec1, vec2))
    return sum_squared ** 0.5


In [5]:
def knn_classify(test_data, train_data, train_labels, k=3):
    distances = []
    for i in range(len(train_data)):
        dist = euclidean_distance(test_data, train_data[i])
        distances.append((dist, train_labels[i]))
    
    distances.sort(key=lambda x: x[0])
    k_nearest = distances[:k]
    
    label_votes = {}
    for _, label in k_nearest:
        if label in label_votes:
            label_votes[label] += 1
        else:
            label_votes[label] = 1
    
    return max(label_votes, key=label_votes.get)


In [ ]:

images, labels, filenames = load_data_from_json_and_images(image_folder, json_file)

# Membagi data 90 : 10
data_size = len(images)
indices = list(range(data_size))

import random
random.shuffle(indices)

train_size = int(data_size * 0.9)
train_indices = indices[:train_size]
test_indices = indices[train_size:]

train_data = [images[i] for i in train_indices]
train_labels = [labels[i] for i in train_indices]
train_filenames = [filenames[i] for i in train_indices]

test_data = [images[i] for i in test_indices]
test_labels = [labels[i] for i in test_indices]
test_filenames = [filenames[i] for i in test_indices]


In [ ]:
k = 3 # Tetangga terdekat
correct_predictions = 0

for i, test_sample in enumerate(test_data):
    predicted_label = knn_classify(test_sample, train_data, train_labels, k)
    
    # Cetak nama file gambar, label prediksi, dan true label
    print(f"Test Image {test_filenames[i]}: Predicted Label = {predicted_label}, True Label = {test_labels[i]}")
    
    # Hitung jumlah prediksi benar
    if predicted_label == test_labels[i]:
        correct_predictions += 1

# Akurasi
accuracy = correct_predictions / len(test_labels) * 100
print(f"Accuracy: {accuracy:.2f}%")


Test Image BEAU4817927-1-_jpg.rf.a5521f10e58def0e78cc22279e28af8b.jpg: Predicted Label = Other, True Label = Other
Test Image CSCU7913029-1-_jpg.rf.1516d08f2115bc7c63d68a356950395f.jpg: Predicted Label = White, True Label = Black
Test Image Container-290_jpg.rf.768c11c822ce7fa77be3f17c52edb7b1.jpg: Predicted Label = White, True Label = White
Test Image Container-703_jpg.rf.b3c9a97de18c700ceda8830a466e2c5b.jpg: Predicted Label = White, True Label = Other
Test Image CSNU7082058-6-_jpg.rf.ed2f758b4efafa5b620c9448b2b59cb2.jpg: Predicted Label = Blue, True Label = Blue
Test Image Container-6_jpg.rf.737cef4ff840080d115b8acb25392735.jpg: Predicted Label = Blue, True Label = Blue
Test Image 2180_990x600c_png_jpg.rf.3a2912510c32e58f3d5519d36db4489f.jpg: Predicted Label = White, True Label = Blue
Test Image 20191030-010538-01-3-R_jpg.rf.64101ecb4dd210bcc1e4d1939ce1fe8e.jpg: Predicted Label = Black, True Label = Black
Test Image 1-123003001-OCR-RF-D01_jpg.rf.4e3eb35ec7c0d0b05cdc9943621208db.jpg: 

In [7]:
# Evaluasi KNN
k = 5 # Tetangga terdekat
correct_predictions = 0

for i, test_sample in enumerate(test_data):
    predicted_label = knn_classify(test_sample, train_data, train_labels, k)
    
    # Cetak nama file gambar, label prediksi, dan label yang benar
    print(f"Test Image {test_filenames[i]}: Predicted Label = {predicted_label}, True Label = {test_labels[i]}")
    
    # Hitung jumlah prediksi benar
    if predicted_label == test_labels[i]:
        correct_predictions += 1

# Akurasi
accuracy = correct_predictions / len(test_labels) * 100
print(f"Accuracy: {accuracy:.2f}%")


Test Image 1-155745001-OCR-LF-C01_jpg.rf.f3281982b59ed1eb7f99f9e06cf333c8.jpg: Predicted Label = Other, True Label = Other
Test Image Container-197_jpg.rf.88679c52eaf2925dcfdd876bc9f2722c.jpg: Predicted Label = Other, True Label = Red
Test Image FCIU988313-4-_638x480_jpg.rf.afa3746759c96bab7f518fc4d0987879.jpg: Predicted Label = Red, True Label = Red
Test Image FBIU0116787-4-_jpg.rf.e926583de69bd8e42768ba9c42720c4d.jpg: Predicted Label = Red, True Label = Red
Test Image 20191029-203832-01-8_jpg.rf.153cca7e615d8b29198a8bad99d08a01.jpg: Predicted Label = Black, True Label = Black
Test Image Container-654_jpg.rf.5f1e37ce34baa797d90d74c515b8c3b9.jpg: Predicted Label = White, True Label = White
Test Image IMG201_jpg.rf.8cc23ad208bb4208dfce62e5080ac816.jpg: Predicted Label = White, True Label = White
Test Image TCKU2683210-3-_jpg.rf.5602276341f98fc3b77756422fb032e7.jpg: Predicted Label = Other, True Label = Blue
Test Image hard_top_shipping_container_vernooy_zeecontainer_428473_7122822_jpg.r